In [1]:
from django_for_jupyter import init_django
init_django()

In [2]:
# tạo dataframe content chứa dữ liệu cho gợi ý dựa trên thể loại
from home.models import Book_Topic, Book
from home.models import ContentBook
import pandas as pd
books = Book.objects.all().order_by('book_id')
bookTopic = Book_Topic.objects.prefetch_related('topic_id').order_by('book_id')
check = True
dics = []
for book in books:
    content = book.book_title
    topics = bookTopic.filter(book_id = book.book_id)
    for topic in topics:
        content+=' '+topic.topic_id.topic_name
    dic = {'book_id': book.book_id,
            'content': content
           }
    dics.append(dic) 
book_df = pd.DataFrame(dics)
print(book_df)

    book_id                                            content
0      3001  CDMA capacity and quality optimization code di...
1      3002  Image databases : Search and retrieval of digi...
2      3003  Practical handbook on image processing for sci...
3      3004  Analytics, data science, & artificial intellig...
4      3005  Composing cyberspace : Identity, community, an...
..      ...                                                ...
74     3075  Digital image processing Image processing Digi...
75     3076  Digital image processing using MATLAB Image pr...
76     3078  Database systems the complete book Database de...
77     3080  Data structures and abstractions with Java Jav...
78     3081  MySQL/PHP database applications database appli...

[79 rows x 2 columns]


In [3]:
from home.models import ContentBook
import pandas as pd
bookContent = ContentBook.objects.all().order_by('book_id')
dics = []
for book in bookContent:
    dic = {
        'book_id': book.book_id,
        'content': book.content
    }
    dics.append(dic)
book_df = pd.DataFrame(dics)
print(book_df)


    book_id                                            content
0      3001  CDMA capacity and quality optimization code di...
1      3002  Image databases : Search and retrieval of digi...
2      3003  Practical handbook on image processing for sci...
3      3004  Analytics, data science, & artificial intellig...
4      3005  Composing cyberspace : Identity, community, an...
..      ...                                                ...
74     3075  Digital image processing Image processing Digi...
75     3076  Digital image processing using MATLAB Image pr...
76     3078  Database systems the complete book Database de...
77     3080  Data structures and abstractions with Java App...
78     3081  MySQL/PHP database applications Database desig...

[79 rows x 2 columns]


In [4]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text


In [5]:
enlish_stop_words = text.ENGLISH_STOP_WORDS
vietnamese_stop_words = ["và", "là", "của", "những", "với", "từ", "một", "được", "khi", "đã", "cho", "vì", "ở", "này", "giáo", "trình", "lập", "trình"]
combine_stop_words = list(enlish_stop_words) + vietnamese_stop_words


In [6]:

book_tfidf = TfidfVectorizer(stop_words=combine_stop_words)
book_df['content'] = book_df['content'].fillna('')
book_content_matrix = book_tfidf.fit_transform(book_df['content'])
book_content_matrix.shape

cosine_similarity = linear_kernel(book_content_matrix, book_content_matrix)

In [7]:
import pickle

# Lưu TF-IDF Vectorizer
with open('./home/recommend/book_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(book_tfidf, f)

with open('./home/recommend/book_cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_similarity, f)

In [8]:
from home.models import ContentBook
import pickle
with open('./home/recommend/book_tfidf_vectorizer.pkl', 'rb') as f:
    book_tfidf = pickle.load(f)
bookContents = ContentBook.objects.all().order_by('book_id')
book_df = pd.DataFrame(bookContents)   
book_content_matrix = book_tfidf.fit_transform(book_df['content'])  

KeyError: 'content'

In [1]:
# Thao tac tinh toan va dua ra goi y
choice = 67
similarity_scores = list(enumerate(cosine_similarity[choice-1]))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
similarity_scores = similarity_scores[1:6]

# Get the similar books index
books_index = [i[0] for i in similarity_scores]
books = Book.objects.all().order_by('book_id')
# printing the top 5 most similar books using integer-location based indexing (iloc)
print(f'The choice is: {books[choice-1].book_id} {books[choice-1].book_title}')
for i in books_index:
    print(f'{books[i].book_id} - {books[i].book_title} ')


NameError: name 'cosine_similarity' is not defined

In [68]:
# Giả sử bạn có dữ liệu về sách mới
new_books_df['content'] = new_books_df['content'].fillna('')

# Vector hóa nội dung sách mới
new_book_content_matrix = book_tfidf.transform(new_books_df['content'])

# Cập nhật ma trận cosine similarity
new_cosine_similarity = linear_kernel(new_book_content_matrix, book_content_matrix)

# Gộp ma trận cũ và mới lại để cập nhật toàn bộ hệ thống
cosine_similarity = linear_kernel(book_content_matrix, book_content_matrix)

# Lưu lại các dữ liệu đã cập nhật
with open('book_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(book_tfidf, f)

with open('book_cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_similarity, f)


In [3]:
from django_for_jupyter import init_django
init_django()

In [143]:
from home.models import ContentBook, Book, Book_Topic
import pandas as pd
latest_book = Book.objects.latest('book_id')
content = latest_book.book_title
topics = Book_Topic.objects.filter(book_id=latest_book.book_id).select_related('topic_id')
for topic in topics:
    content+=' '+ topic.topic_id.topic_name
print(content)
# newContent = ContentBook(book = latest_book, content = content)
# newContent.save()
newContentDic ={
    'book_id': latest_book.book_id,
    'content': content
}
newContent_df = pd.DataFrame(newContentDic, index=[0])
print(newContent_df['content'][0])


Database systems the complete book Database design (Thiết kế database) Database management (Quản trị database)
Database systems the complete book Database design (Thiết kế database) Database management (Quản trị database)


In [9]:
from home.models import ContentBook
from sklearn.metrics.pairwise import linear_kernel
import pickle
import pandas as pd
    
    
with open('./home/recommend/book_tfidf_vectorizer.pkl', 'rb') as f:
    book_tfidf = pickle.load(f)

bookContents = ContentBook.objects.all().order_by('book_id').values('book_id', 'content')
book_df = pd.DataFrame(bookContents)
book_df['content'] = book_df['content'].fillna('')
book_df['content'] = book_df['content'].astype(str)

    
book_content_matrix = book_tfidf.fit_transform(book_df['content']) 
    
cosine_similarity = linear_kernel(book_content_matrix, book_content_matrix)
print(enumerate(cosine_similarity))

In [7]:
from home.models import ContentBook
import pickle
import pandas as pd


bookContents = ContentBook.objects.all().order_by('book_id').values('book_id')
book_df = pd.DataFrame(bookContents)
book_id = 3032
book_df.set_index('book_id', inplace=True)
print(book_df)
book_index = book_df.index.get_loc(book_id)
print(book_index)
with open('./home/recommend/book_cosine_similarity.pkl', 'rb') as f:
    cosine_similarity = pickle.load(f)


Empty DataFrame
Columns: []
Index: [3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008, 3009, 3010, 3011, 3012, 3013, 3014, 3015, 3016, 3017, 3018, 3019, 3020, 3021, 3022, 3023, 3024, 3025, 3026, 3027, 3028, 3029, 3030, 3031, 3032, 3033, 3034, 3035, 3036, 3037, 3038, 3039, 3040, 3041, 3042, 3043, 3044, 3045, 3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063, 3064, 3065, 3066, 3067, 3068, 3069, 3070, 3071, 3072, 3073, 3074, 3075, 3076, 3078, 3079, 3080, 3081]

[80 rows x 0 columns]
31


In [24]:
from home.models import ContentBook
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

books = ContentBook.objects.all().order_by('book_id').values('book_id', 'content')
book_df = pd.DataFrame(books)
book_df['content'] = book_df['content'].fillna('')

enlish_stop_words = text.ENGLISH_STOP_WORDS
vietnamese_stop_words = ["và", "là", "của", "những", "với", "từ", "một", "được", "khi", "đã", "cho", "vì", "ở", "này", "giáo", "trình", "lập", "trình"]
combine_stop_words = list(enlish_stop_words) + vietnamese_stop_words
book_tfidf = TfidfVectorizer(stop_words=combine_stop_words)
book_content_matrix = book_tfidf.fit_transform(book_df['content']) 

cosine_similarities = linear_kernel(book_content_matrix, book_content_matrix)

In [8]:
# Thao tac tinh toan va dua ra goi y


choice = book_index
similarity_scores = list(enumerate(cosine_similarity[choice]))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
similarity_scores = similarity_scores[2:7]

# Get the similar books index
books_index = [i[0] for i in similarity_scores]
from home.models import Book
books = Book.objects.all().order_by('book_id')
# printing the top 5 most similar books using integer-location based indexing (iloc)
print(f'The choice is: {books[choice].book_id} {books[choice].book_title}')
for i in books_index:
    print(f'{books[i].book_id} - {books[i].book_title} ')

The choice is: 3032 Cisco The complete reference
3045 - Giáo trình mạng máy tính 
3013 - Data communications, computer networks and open systems 
3062 - Giáo trình quản trị hệ thống mạng 
3064 - Giáo trình các hệ thống phân tán 
3031 - The complete reference Java 2 


: 

In [14]:
# ham update content book khi update bôok
# lay ra title, topics của cuốn sách mới nhất được cập nhật thông qua created_at
# chạy lại vòng lặp topic và cập nhật content dựa vào book_id
from home.models import ContentBook, Book, Book_Topic
import pandas as pd
lasted_update = Book.objects.latest('created_at')

content = lasted_update.book_title
content = str(content)
topics = Book_Topic.objects.filter(book_id_id = lasted_update.book_id).select_related('topic_id')
for topic in topics:
    print(str(topic.topic_id.topic_name))
    content+=' '+ str(topic.topic_id.topic_name)
    print(content)
content_update = ContentBook.objects.get(book_id = lasted_update.book_id)
content_update.content = content
content_update.save()

updateContentDic = {
    'book_id': lasted_update.book_id,
    'content': content
}
updateContent_df = pd.DataFrame(updateContentDic, index=[0])
print(updateContent_df)


Data structure (cấu trúc dữ liệu)
Perl database programming Data structure (cấu trúc dữ liệu)
Database structure
Perl database programming Data structure (cấu trúc dữ liệu) Database structure
   book_id                                            content
0     3079  Perl database programming Data structure (cấu ...


In [9]:
from home.models import Book
book_id = 3076
bookList = Book.objects.all().order_by('book_id')
detail = bookList.filter(book_id = book_id).first()
print(type(bookList))

<class 'django.db.models.query.QuerySet'>


In [11]:
from home.models import FavList, Book
userID = 1
books = FavList.objects.filter(user_id=userID).select_related('book')
bookList = [fav.book for fav in books]
for book in bookList:
    print(book)

46-Giáo trình kiểm thử phần mềm,Trần, Cao Đệ,Cần Thơ: Nxb. Đại học Cần Thơ, 2012,004.63 / H513,179794,True
48-Giáo trình phân tích hệ thống hướng đối tượng,Phạm, Thị Xuân Lộc,Cần Thơ: Nxb. Đại học Cần Thơ, 2014,004.65 / H516,190026,True
4-Analytics, data science, & artificial intelligence,Sharda, Ramesh,Harlow, United Kingdom: Pearson, 2021,658.403 / S531,244700,True
